In [24]:
import sys
!{sys.executable} -m pip install seaborn


In [25]:
import sys; print(sys.path)


['/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages', '/home/keiseki/My_Python_project/src']


In [26]:
# !pip install japanize-matplotlib
import os
import sys
import yaml
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification                         
import japanize_matplotlib #日本語表示対応


# Notebook から src ディレクトリを追加
sys.path.append("/home/keiseki/My_Python_project/src")

# これで src/preprocess/make_tag_features.py が import 可能
from preprocess.make_tag_features import (
    create_tag_features,
    extract_unique_tags
)

In [27]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=pd.errors.PerformanceWarning
)

## 01.config読み込み

In [28]:
conf_path = os.path.join( '../config/config.yaml')
with open(conf_path, 'r') as f:
    config = yaml.safe_load(f)

In [29]:
# 定義した特徴量リストを読み込み
feature_list = config['FEATURE']['FEATURE_LIST']

In [30]:
# カテゴリカルカラムのリストを定義
categorical_cols = config['FEATURE']['CATEGORICAL_COLS']

## 02.データ読み込み

In [31]:
# ドライブ内のファイルパスを指定
train_path = '../20.Data/train.csv'
df_train = pd.read_csv(train_path)

/tmp/ipykernel_4466/1821816592.py:3: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(train_path)


In [32]:
# testデータの読み込み
test_path = '../20.Data/test.csv'
df_test = pd.read_csv(test_path)

df_test["renovation_date"] = pd.to_datetime(df_test["renovation_date"])
df_test["renovation_date"] = df_test["renovation_date"].astype("int64") // 10**9

/tmp/ipykernel_4466/1294476357.py:3: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(test_path)


## 03.前処理

In [33]:

# df_trainのカテゴリカルカラムを'category'型に変換
for col in categorical_cols:
    if col in df_train.columns:
        df_train[col] = df_train[col].astype('category')

In [34]:
# df_testのカテゴリカルカラムを'category'型に変換
for col in categorical_cols:
    if col in df_test.columns:
        df_test[col] = df_test[col].astype('category')

In [35]:
# 極端に高い売買価格はクリッピング
lower = df_train["money_room"].quantile(0.01)
upper = df_train["money_room"].quantile(0.99)

df_train["money_room"] = df_train["money_room"].clip(lower, upper)


In [36]:
df_train["land_youto"] = df_train["land_youto"].where(
    df_train["land_youto"].isin([8, 14]),
    np.nan
).astype("category")

df_test["land_youto"] = df_test["land_youto"].where(
    df_test["land_youto"].isin([8, 14]),
    np.nan
).astype("category")


In [ ]:
df_train["building_type"] = (
    df_train["building_type"]
    .where(df_train["building_type"].isin([1, 3]), np.nan)
).astype("category")

df_test["building_type"] = (
    df_test["building_type"]
    .where(df_test["building_type"].isin([1, 3]), np.nan)
).astype("category")


In [38]:
# 欠損値はカラムの型に応じて補完
for col in df_train.columns:
    if col == "money_room": # 目的変数はこの時点では補完しない
        continue

    # if col in categorical_cols: # 明示的に定義されたカテゴリカルカラム
    #     # LightGBMが0をカテゴリとして扱えるため、0で補完
    #     df_train[col] = df_train[col].fillna(0)
    elif df_train[col].dtype in ['int64', 'float64']: # 数値カラム
        # 数値カラムは訓練データの中央値で補完
        median_val = df_train[col].median()
        df_train[col] = df_train[col].fillna(median_val)
        df_test[col] = df_test[col].fillna(median_val)
    # else:
        # # その他の型（objectなど）は、とりあえず0で補完（必要に応じて調整）
        # df_train[col] = df_train[col].fillna(0)


/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/keiseki/My_Python_project/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/keiseki/M

In [39]:
# # room_count
# # unit_area
df_train = df_train[(df_train["unit_area"]>12.5) & (df_train["unit_area"]<300)]

##### 日付関係のデータは経過日数を特徴量とする

In [40]:
# 'target_ym'をdatetime型に変換 (月の最初の日と仮定)
df_train['target_date'] = pd.to_datetime(df_train['target_ym'].astype(str) + '01', format='%Y%m%d')
df_test['target_date'] = pd.to_datetime(df_test['target_ym'].astype(str) + '01', format='%Y%m%d')

# 'building_create_date'をdatetime型に変換
df_train['building_create_date'] = pd.to_datetime(df_train['building_create_date'])
df_test['building_create_date'] = pd.to_datetime(df_test['building_create_date'])
# 築年数を計算 (年単位)
df_train['building_age'] = (df_train['target_date'] - df_train['building_create_date']).dt.days // 365
df_test['building_age'] = (df_test['target_date'] - df_test['building_create_date']).dt.days // 365

# 'renovation_date'がUNIXタイムスタンプになっているため、再度datetimeに変換して経過年数を計算
# 既に'renovation_date'はUNIXタイムスタンプに変換済みのため、元の日付情報に戻すか、元のカラムを使用する
# ここでは、元のカラムが保持されていると仮定し、新しいカラムとして計算
# 注: もしdf_train['renovation_date']がUNIXタイムスタンプのみになっている場合、この処理は調整が必要です。
# 既存のrenovation_date（UNIXタイムスタンプ）を無視し、元の文字列カラムが存在しないため、新たに変換する
# もし元の文字列カラムが失われている場合は、UNIXタイムスタンプをdatetimeに変換する
# 現状のNotebookの処理から判断すると、renovation_dateはUNIXタイムスタンプなので、それを基に経過年数を計算します。

# UNIXタイムスタンプからdatetimeへの変換
df_train['renovation_datetime'] = pd.to_datetime(df_train['renovation_date'], errors='coerce')
df_test['renovation_datetime'] = pd.to_datetime(df_test['renovation_date'], errors='coerce')

# リノベーションからの経過年数を計算 (年単位)
# リノベーション日がない場合はNaNとなり、NaNとtarget_dateの差もNaNになるため、後でfillnaで処理
df_train['years_since_renovation'] = (df_train['target_date'] - df_train['renovation_datetime']).dt.days // 365
df_test['years_since_renovation'] = (df_test['target_date'] - df_test['renovation_datetime']).dt.days // 365
# 欠損値の補完
# building_ageとyears_since_renovationの負の値（未来の日付）やNaNを処理
# 例: 負の値やNaNを0で補完する
df_train['building_age'] = df_train['building_age'].apply(lambda x: max(0, x) if pd.notna(x) else 0)
df_test['building_age'] = df_test['building_age'].apply(lambda x: max(0, x) if pd.notna(x) else 0)

df_train['years_since_renovation'] = df_train['years_since_renovation'].apply(lambda x: max(0, x) if pd.notna(x) else 0)
df_test['years_since_renovation'] = df_test['years_since_renovation'].apply(lambda x: max(0, x) if pd.notna(x) else 0)
# 新しい特徴量をfeature_listに追加
new_features = ['building_age', 'years_since_renovation']
feature_list.extend(new_features)

# feature_listから'target_date', 'building_create_date', 'renovation_datetime'を削除 (直接特徴量として使わないため)
# (元のbuilding_create_dateとrenovation_dateはUNIX timestampとして残しておく)
feature_list = [f for f in feature_list if f not in ['target_date', 'building_create_date', 'renovation_date', 'renovation_datetime']]

# 確認
# print(df_train[feature_list].head())



## 04.特徴量生成

In [41]:
# addr1_1 と addr2_1をあわせた特徴量を作成
df_train['addr1+2'] = df_train['addr1_1'].astype(str) + '_' + df_train['addr1_2'].astype(str)
df_test['addr1+2'] = df_test['addr1_1'].astype(str) + '_' + df_test['addr1_2'].astype(str)
feature_list.append('addr1+2')

In [42]:
# タグIDの分割により特徴量を追加

# all_unique_tags という空の set を初期化
all_unique_tags = set()
# df_train の 'statuses' カラムに対して extract_unique_tags を呼び出し、得られたユニークタグを追加
all_unique_tags.update(extract_unique_tags(df_train, 'statuses'))
# df_test の 'statuses' カラムに対して extract_unique_tags を呼び出し、得られたユニークタグを追加
all_unique_tags.update(extract_unique_tags(df_test, 'statuses'))
# all_unique_tags をソートしてリストに変換し、all_unique_tags_list に格納
all_unique_tags_list = sorted(list(all_unique_tags))

# create_tag_features 関数を df_train と df_test に適用
df_train, new_tag_features = create_tag_features(df_train.copy(), all_unique_tags_list)
df_test, _ = create_tag_features(df_test.copy(), all_unique_tags_list)
# 既存の feature_list に new_tag_features を追加
feature_list.extend(new_tag_features)

KeyboardInterrupt: 

In [ ]:
# 極端な値はクリッピング
for col in feature_list:
    # 数値型でなければスキップ
    if not pd.api.types.is_numeric_dtype(df_train[col]):
        continue

    # 全部NaNの列はスキップ
    if df_train[col].notna().sum() == 0:
        continue

    # 1% / 99% パーセンタイル
    lower = df_train[col].quantile(0.01)
    upper = df_train[col].quantile(0.99)

    # パーセンタイルが計算できない場合はスキップ
    if pd.isna(lower) or pd.isna(upper):
        continue

    # クリッピング
    df_train[col] = df_train[col].clip(lower, upper)


In [ ]:
# LightGBM 用に使用するカテゴリ特徴量のリストを作成
categorical_features = [
    col for col in categorical_cols
    if col in feature_list
]

feature_list = feature_list.copy()

In [ ]:
df_train['addr1+2'] = df_train['addr1+2'].astype('category')
df_test['addr1+2']  = df_test['addr1+2'].astype('category')


＿前処理後のDataFrameを出力保存

In [ ]:
# 前処理後のDataFrameをpklファイルで出力保存

df_train.to_pickle('../20.Data/processed_train.pkl')
df_test.to_pickle('../20.Data/processed_test.pkl')